# Correction exercice sur Spacy

« À l'aide de la bibliothèque Spacy vous relèverez les personnages
mentionnés dans Le ventre de Paris.
Pour chacun des personnages qui apparaissent au moins trois fois, vous
indiquerez les verbes dont ils sont sujet. »

Il y a plusieurs façons de faire. Ici nous allons utiliser `Dependency Matcher` et tenter d'utiliser `displacy` pour visualiser les résultats.

In [1]:
import spacy
from spacy.matcher import DependencyMatcher

# Nous allons stocker les résultats dans l'objet Doc, dans un nouvel attribut
# spacy a prévu un mécanisme d'extension pour ça
from spacy.tokens import Doc
Doc.set_extension("my_entities", default=True)

nlp = spacy.load('fr_core_news_md')

In [5]:
doc = ""
with open('files/Le_Ventre_de_Paris-chap1.txt') as input_f:
    doc = nlp(input_f.read())
doc._.my_entities = [] # on ajoute la liste my_entities comme une extension à l'objet Doc

In [6]:
def get_dep(token, dep_type="flat:name"):
    """
    Cherche et retourne tous les tokens gouvernés
    par 'token' avec la dépendance passée en paramètre
    Par défaut 'flat:name' pour récupérer "Madame François"
    plutôt que juste "Madame"
    """
    res = []
    res.append(token)
    for child in token.children:
        if child.dep_ == dep_type:
            res.append(child)
    return res

In [7]:
def add_persverb_event(matcher, doc, id, matches):
    """
    Fonction callback de la méthode `matcher.add`
    voir https://spacy.io/api/dependencymatcher#add
    Ajoute une entité type 'PERSVERB' au couple sujet verbe détecté
    """
    for m_id, t_ids in matches:
        flats = get_dep(doc[t_ids[1]]) # le personnage et ses dépendants de type 'flat:name'
        verb = doc[t_ids[0]]
        # les entités sont stockées au format demandé pour une utilisation manuelle
        # voir https://spacy.io/usage/visualizers#manual-usage
        pers = {'start':flats[0].idx, 'end':flats[-1].idx+len(flats[-1])+1, 'label':"PERS"}
        verb = {'start':verb.idx, 'end':verb.idx+len(verb)+1, 'label':'VERB'}
        doc._.my_entities.extend([pers, verb])

In [8]:
matcher = DependencyMatcher(nlp.vocab)
pattern = [
    {
        "RIGHT_ID": "verb",    
        "RIGHT_ATTRS": {"POS": "VERB"}
    },
     {
        "LEFT_ID": "verb",
        "REL_OP": ">",
        "RIGHT_ID": "pers",
        "RIGHT_ATTRS": {"DEP": "nsubj", "ENT_TYPE": "PER"},  
    }
]
matcher.add("pers-verb", [pattern], on_match=add_persverb_event)

In [9]:
matches = matcher(doc)

In [10]:
from spacy import displacy

content = [
    {'text': doc.text,
    'ents': doc._.my_entities, # on retrouve les entités stockée en extension du Doc
    'title': 'Personnages - verbes' 
    }
]
# export dans un fichier html
# pour un roman entier le poids du fichier sera peut-être trop important
html = displacy.render(content, style="ent", manual=True, jupyter=False, minify=True)
with open('./personnages-verbes.html', 'w') as output:
    output.write(html)

Ça ne répond pas exactement à la question puisqu'il fallait ne retenir que les personnages qui apparaissent au moins trois fois.  
Pour ça on va travailler à partir de l'objet `matches`, résultat du `Dependancy Matcher`.

In [11]:
from collections import defaultdict

res = defaultdict(list)

for m_id, t_ids in matches:
        flats = get_dep(doc[t_ids[1]]) # le personnage et ses dépendants de type 'flat:name'
        sujet = " ".join([token.text for token in flats]) # on récupère la chaîne de caractères
        verb = doc[t_ids[0]]
        res[sujet].append(verb.text) # chaque verbe sera ajouté à la liste du personnage concerné

for sujet in res:
    if len(res[sujet]) > 2:
        print(f"- {sujet} : {', '.join(res[sujet])}")

- Madame François : regardait, pensa, causait
- Florent : songeait, avançait, entra, mit, donné, aperçut, regardait, rappelait, aperçut, causait, intéressa, étonnait, regardait, allait, dit, regardait, levait, appuyer, écoutait, savait, tournait, demanda, crut, regardait, imagina, heurtait, raconta, promené, avait, entrât, entendit, reconnaissait, revit
- Claude : regardait, arrêté, riait, dit, appela, entra, appelait, répéta, dit, ravi, dit, racontait, regardait, montra, monté, battait, murmurait
